# Tutorial 05 - Gaussian

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import numpy.typing
import petsc4py.PETSc
import ufl
import viskex

In [ ]:
import rbnicsx.backends

## 1. Mesh generation

In [ ]:
mesh_size = 1e-1

In [ ]:
gmsh.initialize()
gmsh.model.add("gaussian")

In [ ]:
p0 = gmsh.model.geo.addPoint(-1.0, -1.0, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(1.0, -1.0, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(1.0, 1.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(-1.0, 1.0, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
boundary = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])
domain = gmsh.model.geo.addPlaneSurface([boundary])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1, l3], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(2, [domain], 1)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)
gmsh.finalize()

In [ ]:
viskex.dolfinx.plot_mesh(mesh)

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, subdomains, "subdomains")

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, boundaries, "boundaries")

## 2. Problem definition

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.functionspace(mesh, ("Lagrange", 1))
        self._V = V
        # Define trial and test functions
        u = ufl.TrialFunction(V)
        v = ufl.TestFunction(V)
        # Define solution
        solution = dolfinx.fem.Function(V)
        self._solution = solution
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(2, ))
        self._mu_symb = mu_symb
        # Define bilinear form of the problem
        a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        # Define linear form of the problem
        x = ufl.SpatialCoordinate(mesh)
        g = ufl.exp(- 2 * (x[0] - mu_symb[0])**2 - 2 * (x[1] - mu_symb[1])**2)
        f = ufl.inner(g, v) * ufl.dx
        self._f = f
        self._f_cpp = dolfinx.fem.form(f)
        # Define boundary conditions for the problem
        zero = petsc4py.PETSc.ScalarType(0)
        bdofs_V = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, boundaries.indices)
        bcs = [dolfinx.fem.dirichletbc(zero, bdofs_V, V)]
        self._bcs = bcs

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpaceBase:
        """Return the function space of the problem."""
        return self._V

    @property
    def bilinear_form(self) -> ufl.Form:  # type: ignore[no-any-unimported]
        """Return the bilinear form of the problem."""
        return self._a

    @property
    def linear_form(self) -> ufl.Form:  # type: ignore[no-any-unimported]
        """Return the linear form of the problem."""
        return self._f

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBC]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:  # type: ignore[no-any-unimported]
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.petsc.assemble_matrix(self._a_cpp, bcs=self._bcs)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:  # type: ignore[no-any-unimported]
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.petsc.assemble_vector(self._f_cpp)
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.petsc.set_bc(F, self._bcs)
        return F

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value and solve the problem."""
        self._mu_symb.value[:] = mu
        return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = self._solution.copy()
        ksp.solve(F, solution.vector)
        solution.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        A.destroy()
        F.destroy()
        ksp.destroy()
        return solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([0.3, -1.0])
solution = problem.solve(mu_solve)

In [ ]:
viskex.dolfinx.plot_scalar_field(solution, "high fidelity solution")